In [1]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.utils import save_image
import matplotlib.pyplot as plt

In [2]:
EPOCHS = 500
BATCH_SIZE=100
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print('다음 장치를 사용합니다:', DEVICE)

다음 장치를 사용합니다: cpu


In [3]:
trainset = datasets.FashionMNIST('./.data',
                                train=True,
                                download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5,),(0.5,))
                                ]))
train_loader = torch.utils.data.DataLoader(
    dataset = trainset,
    batch_size = BATCH_SIZE,
    shuffle = True
)

In [4]:
# Generator
G = nn.Sequential(    # Sequential : 신경망을 이루는 각 layer에서 수행할 연산들을 입력받아 순서대로 실행
        nn.Linear(64, 256),   # 정규분포로부터 뽑은 64차원의 무작위 tensor
        nn.ReLU(),
        nn.Linear(256, 256),
        nn.ReLU(),
        nn.Linear(256, 784),
        nn.Tanh()     # Tanh 함수를 통해 -1~1의 값을 가짐
)

In [5]:
# Discriminator
D = nn.Sequential(
        nn.Linear(784, 256),    # 이미지의 크기는 28*28=784
        nn.LeakyReLU(0.2),      # ReLU가 아니라 LeakyReLU를 사용하는 이유 : 양의 기울기만 갖지 않고 약간의 음의 기울기도 갖기 때문에 생성자에 더 강하게 전달됨 (생성자가 학습 시 판별자로부터 기울기를 효과적으로 전달받는 것이 중요)
        nn.Linear(256, 256),
        nn.LeakyReLU(0.2),
        nn.Linear(256, 1),
        nn.Sigmoid()      # Sigmoid 함수를 통해 0~1의 값을 가짐 (0=가짜, 1=진짜)
)

In [6]:
D = D.to(DEVICE)
G = G.to(DEVICE)

In [7]:
# 오차 함수 : 이진 교차 엔트로피 
criterion = nn.BCELoss()    

# 최적화 알고리즘 : Adam
d_optimizer = optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = optim.Adam(G.parameters(), lr=0.0002)

In [ ]:
total_step = len(train_loader)

for epoch in range(EPOCHS):  # GAN 학습
    for i, (images, _) in enumerate(train_loader):
        images = images.reshape(BATCH_SIZE, -1).to(DEVICE)
        
        real_labels = torch.ones(BATCH_SIZE, 1).to(DEVICE)
        fake_labels = torch.zeros(BATCH_SIZE, 1).to(DEVICE)
        
        # discriminator가 진짜 이미지를 진짜로 인식하는 오차 계산
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs

        # generator는 무작위 텐서로 가짜 이미지 생성
        z = torch.randn(BATCH_SIZE, 64).to(DEVICE)   # 무작위 텐서
        fake_images = G(z)

        # discriminator가 가짜 이미지를 가짜로 인식하는 오차 계산
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs

        # discriminator의 오차 구함
        d_loss = d_loss_real + d_loss_fake

        # discriminator 신경망 학습
        d_optimizer.zero_grad()   # 기울기 초기화
        g_optimizer.zero_grad()
        d_loss.backward()   # 역전파 알고리즘
        d_optimizer.step()

        fake_images = G(z)
        outputs = D(fake_images)
        g_loss = criterion(outputs, real_labels)  # generator가 discriminator를 속였는지에 대한 오차 계산

        # generator 학습 (가짜를 진짜로 인식하도록)
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
        
    # 학습 진행 알아보기
    # D(x): 진짜를 진짜로 인식, D(G(z)): 가짜를 진짜로 인식
    print('epoch [{}/{}] d_loss:{:.4f} g_loss:{:.4f} D(x):{:.2f} D(G(z)):{:.2f}'.format(epoch, EPOCHS, d_loss.item(), g_loss.item(), real_score.mean().item(), fake_score.mean().item()))

epoch [0/500] d_loss:0.0415 g_loss:4.5327 D(x):0.98 D(G(z)):0.02
epoch [1/500] d_loss:0.0149 g_loss:6.3717 D(x):0.99 D(G(z)):0.01
epoch [2/500] d_loss:0.0899 g_loss:7.1315 D(x):0.96 D(G(z)):0.00
epoch [3/500] d_loss:0.0649 g_loss:6.2821 D(x):0.97 D(G(z)):0.02
epoch [4/500] d_loss:0.0677 g_loss:4.9087 D(x):0.98 D(G(z)):0.04
epoch [5/500] d_loss:0.1075 g_loss:5.3211 D(x):0.96 D(G(z)):0.02
epoch [6/500] d_loss:0.0934 g_loss:6.1173 D(x):0.96 D(G(z)):0.02
epoch [7/500] d_loss:0.2204 g_loss:4.1025 D(x):0.93 D(G(z)):0.06
epoch [8/500] d_loss:0.1173 g_loss:6.9752 D(x):0.96 D(G(z)):0.04
epoch [9/500] d_loss:0.1810 g_loss:4.8100 D(x):0.95 D(G(z)):0.05
epoch [10/500] d_loss:0.1656 g_loss:4.7332 D(x):0.93 D(G(z)):0.06
epoch [11/500] d_loss:0.3498 g_loss:5.4743 D(x):0.89 D(G(z)):0.03
epoch [12/500] d_loss:0.4986 g_loss:3.6416 D(x):0.90 D(G(z)):0.13
epoch [13/500] d_loss:0.2413 g_loss:3.7221 D(x):0.95 D(G(z)):0.09
epoch [14/500] d_loss:0.2135 g_loss:4.7553 D(x):0.93 D(G(z)):0.05
epoch [15/500] d_los

In [ ]:
# generator가 만들어낸 가짜 이미지 시각화
z = torch.randn(BATCH_SIZE, 64).to(DEVICE)
fake_images = G(z)

for i in range(10):
    fake_images_img = np.reshape(fake_images.data.cpu().numpy()[i], (28,28))
    plt.imshow(fake_images_img, cmap = 'gray')
    plt.show()